In [6]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git
import numpy as np
import torch
from pkg_resources import packaging
import clip
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from collections import OrderedDict
import torch
import skimage
import os
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import torchvision
from torchvision.datasets import EuroSAT
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from torch.utils.data import DataLoader
import random

print("Torch version:", torch.__version__)
model, preprocess = clip.load("RN50") #The model variable will hold the loaded model, while the preprocess variable will store the preprocessing function for preparing the input data.
#model.load_state_dict(torch.load('/home/xz306/Data-Climate-and-AI/src/fine_tuning/try/clip-roberta-finetuned/pytorch_model.bin'))
model.cuda().eval()
input_resolution = model.visual.input_resolution # (size) of the input images expected by the model.
context_length = model.context_length # the length of the tokenized text input that the model can process.
vocab_size = model.vocab_size # the size of the vocabulary used by the model.

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Torch version: 2.0.1+cu117
Model parameters: 102,007,137
Input resolution: 224
Context length: 77
Vocab size: 49408


Main code to test the performance of original CLIP on EuroSAT


In [29]:
models = ['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']
for k in range(len(models)):
    model, preprocess = clip.load(models[k])
    model.cuda().eval()
    Eurosat = EuroSAT(root='~/.cache', transform=preprocess, download=True)
    classes = Eurosat.classes
    classes[0] = 'annual crop land'
    classes[2] = 'brushland or shrubland'
    classes[4] = 'highway or road'
    classes[5] = 'pasture land'
    classes[6] = 'permanant crop land'
    dataloader = DataLoader(Eurosat, batch_size=32, shuffle=True)
    device = "cuda"
    count = 0
    top_k_accuracy = 1
    count_total = 0
    for i, (images, labels) in enumerate(dataloader):   
        label = []
        # Iterate over the dataset and store images and labels
        for j in range(len(labels)):
        # Retrieve image and label at index i
            label.append(classes[labels[j]])
        with torch.no_grad():
            image_input = torch.tensor(np.stack(images)).cuda()
            text_descriptions = [f"a centered satellite photo of a {labe}" for labe in classes]
            text_tokens = clip.tokenize(text_descriptions).cuda()
            image_features = model.encode_image(image_input).float() # image encoder
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features = model.encode_text(text_tokens).float() # text encoder
            text_features /= text_features.norm(dim=-1, keepdim=True)
        text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)   
        for i in range(len(label)):
            count_total += 1
            values, indices = text_probs[i].topk(top_k_accuracy)
            for value, index in zip(values, indices):
                #print (Eurosat.classes[index])
                if Eurosat.classes[index] == label[i]:
                    count += 1
                    break
    print(models[k])
    print ("Accuracy = ", count/len(Eurosat))


ViT-B/32
Accuracy =  0.3382222222222222




Code to test the performance of tuned model on EuroSAT. Change the paths of model and processor to where you save your saved model.

In [8]:
#model.cuda().eval()
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel
import transformers
from transformers import (
    VisionTextDualEncoderModel,
    VisionTextDualEncoderProcessor,
    AutoTokenizer,
    AutoImageProcessor
)
import torch
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor

from transformers import CLIPProcessor, CLIPModel
import numpy as np
import torch
from pkg_resources import packaging
import clip
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from collections import OrderedDict
import torch
import skimage
import os
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import torchvision
from torchvision.datasets import EuroSAT
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from torch.utils.data import DataLoader
import random
# seed = 42
# torch.manual_seed(seed)
# random.seed(seed)
Eurosat = EuroSAT(root='~/.cache', download=True)
classes = Eurosat.classes
Eurosat = list(Eurosat)
random.shuffle(Eurosat)
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

classes[0] = 'annual crop land'
classes[2] = 'brushland or shrubland'
classes[4] = 'highway or road'
classes[5] = 'pasture land'
classes[6] = 'permanant crop land'
text = []
count = 0
for i in range(len(classes)):
    text.append('a satellite photo of' + classes[i])
labels = []
images = []
# Iterate over the dataset and store images and labels
for i in range(100):
    images.append(Eurosat[i][0])
    labels.append(Eurosat[i][1])
inputs = processor(images=images, text=text,  return_tensors="pt", padding=True)
outputs = model(**inputs)   
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)
max_indices = torch.argmax(probs, dim=1)
result = max_indices.tolist()
count_same_values = 0
for i in range(min(len(result), len(labels))):
    if result[i] == labels[i]:
        count_same_values += 1
accuracy = count_same_values/100
print(accuracy)

0.37
